In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
db_password = os.getenv("DB_PASSWORD")

In [ ]:
from langchain_community.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username ="neo4j"
password = db_password

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [ ]:
import_query = """
LOAD CSV FROM 'file:///relations.csv' AS row
MERGE (e:Person {head: row[1], tail: row[3], relation: row[2]})
RETURN e.head, e.tail, e.relation
"""
graph.query(
    import_query
)

In [ ]:
import os
from dotenv import load_dotenv

api_key = os.getenv("API_KEY")

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(openai_api_key=api_key),
    url=url,
    username=username,
    password=password,
    index_name='person',
    node_label="Person",
    text_node_properties=['head', 'tail', 'relation'],
    embedding_node_property='embedding',
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff", retriever=vector_index.as_retriever())

In [ ]:
vector_qa.invoke(
    {"query": "How many pools did the San Antonio Parks and Recreation open?"}
)

In [ ]:
result = vector_qa.invoke(
    {"query": "What did the San Antonio Parks and Recreation announce?"}
)

print(result['result'])